## 分块

In [ ]:
from typing import List
from langchain_experimental.text_splitter import SemanticChunker
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

text_splitter = RecursiveCharacterTextSplitter()

hf_embedder = HuggingFaceEmbeddings(model_name="BAAI/bge-base-zh")
semantic_text_splitter = SemanticChunker(hf_embedder)

useSemantic = False

def split_to_chunks(file_path: str) -> List[str]:
    with open(file_path, 'r') as file:
        content = file.read()

    splitter = text_splitter if not useSemantic else semantic_text_splitter
    docs = splitter.create_documents([content])
    chunks = [doc.page_content for doc in docs]

    return [chunk.strip() for chunk in chunks if chunk.strip()]

chunks = split_to_chunks('sanguo.txt')
print(f"Total chunks: {len(chunks)}")

# for i, chunk in enumerate(chunks[:10]):
#     print(f"Chunk {i+1}: {chunk}\n")

/Users/onlyil/Documents/Personal/Projects/ai-practices/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Total chunks: 164


## 向量化

In [11]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('BAAI/bge-base-zh')

def get_embeddings(text: str) -> List[float]:
    return embedder.encode(text).tolist()

embeddings = [get_embeddings(chunk) for chunk in chunks]

## 持久化

In [12]:
import chromadb
import numpy as np

client = chromadb.PersistentClient()
collection = client.get_or_create_collection("rag")

def save_embeddings(chunks: List[str], embeddings: List[List[float]]):
    collection.add(
        documents=chunks,
        embeddings=np.array(embeddings, dtype=np.float32),
        ids=[str(i) for i in range(len(chunks))]
    )

save_embeddings(chunks, embeddings)

## 召回

In [ ]:
def retrieve(query: str, top_k: int) -> List[str]:
    query_embedding = get_embeddings(query)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )

    if not results['documents']:
        return []

    return results['documents'][0]

# query = "谁直呼黛玉神女下凡？"
query = "张飞的武器是什么？"
retrieved_chunks = retrieve(query, 5)
print(retrieved_chunks)

# for i, chunk in enumerate(retrieved_chunks):
#     print(f"Retrieved Chunk {i+1}: {chunk}\n")

## 重排

In [ ]:
from sentence_transformers import CrossEncoder

def rerank(query: str, retrieved_chunks: List[str], top_k: int) -> List[str]:
    cross_encoder = CrossEncoder('BAAI/bge-reranker-base')
    pairs = [[query, chunk] for chunk in retrieved_chunks]
    scores = cross_encoder.predict(pairs)
    chunk_with_scores = list(zip(retrieved_chunks, scores))
    chunk_with_scores.sort(key=lambda x: x[1], reverse=True)
    return [chunk for chunk, _ in chunk_with_scores[:top_k]]

reranked_chunks = rerank(query, retrieved_chunks, 3)

# for i, chunk in enumerate(reranked_chunks):
#     print(f"Reranked Chunk {i+1}: {chunk}\n")

Reranked Chunk 1: 先主大怒曰："朕自离成都许多时，你两个如何不来请罪？今日势危，故来巧言，欲全性命！朕若饶你，至九泉之下，有何面目见关公乎！"言讫，令关兴在御营中，设关公灵位。先主亲捧马忠首级，诣前祭祀。又令关兴将糜芳、傅士仁剥去衣服，跪于灵前，亲自用刀剐之，以祭关公。忽张苞上帐哭拜于前曰："二伯父仇人皆已诛戮；臣父冤仇，何日可报？”

先主曰："贤侄勿忧。朕当削平江南，杀尽吴狗，务擒二贼，与汝亲自醢之，以祭汝父。"

苞泣谢而退。
此时先主威声大震，江南之人尽皆胆裂，日夜号哭。韩当、周泰大惊，急奏吴王，具言糜芳、傅士仁杀了马忠，去归蜀帝，亦被蜀帝杀了。孙权心怯，遂聚文武商议。步骘奏曰："蜀主所恨者，乃吕蒙、潘璋、马忠、糜芳、傅士仁也。今此数人皆亡，独有范疆、张达二人，现在东吴。何不擒此二人，并张飞首级，遣使送还，交与荆州，送归夫人，上表求和，再会前情，共图灭魏，则蜀兵自退矣。"

权从其言，遂具沉香木匣，盛贮飞首，绑缚范疆、张达，囚于槛车之内，令程秉为使，赍国书，望猇亭而来。
却说先主欲发兵前进。忽近臣奏曰："东吴遣使送张车骑之首，并囚范疆、张达二贼至。"

先主两手加额曰："此天之所赐，亦由三弟之灵也！"即令张苞设飞灵位。先主见张飞首级在匣中面不改色，放声大哭。张苞自仗利刀，将范疆、张达万剐凌迟，祭父之灵。祭毕，先主怒气不息，定要灭吴。马良奏曰："仇人尽戳，其恨可雪矣。吴大夫程秉到此，欲还荆州，送回夫人，永结盟好，共图灭魏，伏候圣旨。"

先主怒曰："朕切齿仇人，乃孙权也。今若与之连和，是负二弟当日之盟矣。今先灭吴，次灭魏。"

便欲斩来使，以绝吴情。多官苦告方免。程秉抱头鼠窜，回奏吴主曰："蜀不从讲和，誓欲先灭东吴，然后伐魏。众臣苦谏不听，如之奈何？”

权大惊，举止失措。阚泽出班奏曰："现有擎天之柱，如何不用耶？”

权急问何人。泽曰："昔日东吴大事，全任周郎；后鲁子敬代之；子敬亡后，决于吕子明；今子明虽丧，现有陆伯言在荆州。此人名虽儒生，实有雄才，大略，以臣论之，不在周郎之下；前破关公，其谋皆出于伯言。主上若能用之，破蜀必矣。如或有失，臣愿与同罪。"

权曰："非德润之言，孤几误大事。"

张昭曰："陆逊乃一书生耳，非刘备敌手；恐不可用。"

顾雍亦曰："陆逊年幼望轻，恐诸公不服；若不服则生祸乱，必误大事。"

来骘亦曰："逊才堪治郡耳；

## LLM

In [15]:
from google import genai
from dotenv import load_dotenv

load_dotenv()

client = genai.Client()

def generate_answer(query: str, context: List[str]):
    prompt = f"根据以下内容回答问题：\n\n{''.join(context)}\n\n问题：{query}\n请基于以上内容作答，若无法从中获取答案，请回复“无法从提供的内容中获取答案”。"
    print(f"Prompt: {prompt}\n")
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt
    )
    return response.text

answer = generate_answer(query, reranked_chunks)
print(f"Answer: {answer}\n")

Prompt: 根据以下内容回答问题：

先主大怒曰："朕自离成都许多时，你两个如何不来请罪？今日势危，故来巧言，欲全性命！朕若饶你，至九泉之下，有何面目见关公乎！"言讫，令关兴在御营中，设关公灵位。先主亲捧马忠首级，诣前祭祀。又令关兴将糜芳、傅士仁剥去衣服，跪于灵前，亲自用刀剐之，以祭关公。忽张苞上帐哭拜于前曰："二伯父仇人皆已诛戮；臣父冤仇，何日可报？”

先主曰："贤侄勿忧。朕当削平江南，杀尽吴狗，务擒二贼，与汝亲自醢之，以祭汝父。"

苞泣谢而退。
此时先主威声大震，江南之人尽皆胆裂，日夜号哭。韩当、周泰大惊，急奏吴王，具言糜芳、傅士仁杀了马忠，去归蜀帝，亦被蜀帝杀了。孙权心怯，遂聚文武商议。步骘奏曰："蜀主所恨者，乃吕蒙、潘璋、马忠、糜芳、傅士仁也。今此数人皆亡，独有范疆、张达二人，现在东吴。何不擒此二人，并张飞首级，遣使送还，交与荆州，送归夫人，上表求和，再会前情，共图灭魏，则蜀兵自退矣。"

权从其言，遂具沉香木匣，盛贮飞首，绑缚范疆、张达，囚于槛车之内，令程秉为使，赍国书，望猇亭而来。
却说先主欲发兵前进。忽近臣奏曰："东吴遣使送张车骑之首，并囚范疆、张达二贼至。"

先主两手加额曰："此天之所赐，亦由三弟之灵也！"即令张苞设飞灵位。先主见张飞首级在匣中面不改色，放声大哭。张苞自仗利刀，将范疆、张达万剐凌迟，祭父之灵。祭毕，先主怒气不息，定要灭吴。马良奏曰："仇人尽戳，其恨可雪矣。吴大夫程秉到此，欲还荆州，送回夫人，永结盟好，共图灭魏，伏候圣旨。"

先主怒曰："朕切齿仇人，乃孙权也。今若与之连和，是负二弟当日之盟矣。今先灭吴，次灭魏。"

便欲斩来使，以绝吴情。多官苦告方免。程秉抱头鼠窜，回奏吴主曰："蜀不从讲和，誓欲先灭东吴，然后伐魏。众臣苦谏不听，如之奈何？”

权大惊，举止失措。阚泽出班奏曰："现有擎天之柱，如何不用耶？”

权急问何人。泽曰："昔日东吴大事，全任周郎；后鲁子敬代之；子敬亡后，决于吕子明；今子明虽丧，现有陆伯言在荆州。此人名虽儒生，实有雄才，大略，以臣论之，不在周郎之下；前破关公，其谋皆出于伯言。主上若能用之，破蜀必矣。如或有失，臣愿与同罪。"

权曰："非德润之言，孤几误大事。"

张昭曰："陆逊乃一书生耳，非刘备敌手；恐不可用。"

顾雍亦曰："陆逊年幼望轻，恐诸公不服；若不服则生祸乱，必误大事。"

来骘亦曰："逊才堪治